In [1]:
import numpy as np
import pandas as pd
import numbers
from copy import *
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
import seaborn as sns
import missingno as msno
import sklearn.metrics as metrics
plt.style.use('ggplot')
sns.set_style('whitegrid')

# TODO
pd.set_option('display.max_columns', None)
#data = pd.read_csv("clinvar_conflicting.csv")
data = pd.read_table("clinvar_conflicting_1.txt",sep=',')
data1 = deepcopy(data)

data1

ModuleNotFoundError: No module named 'missingno'

In [ ]:
msno.bar(data1,color='#FDBB22',sort='descending')
plt.show()

In [ ]:
data1.shape

In [ ]:
total = len(data1)
ax = sns.countplot(x = 'CLASS', data=data1 ,palette="icefire")
plt.title('Classification Conflicts in Dataset', fontsize=20, y=1.08)
sns.set(font_scale=1)
ax.set_xlabel('Target Label Distribution')
ax.set_ylabel('Number of Total Variants')
fig = plt.gcf()
fig.set_size_inches(12,5)
ax2=ax.twinx()
ax2.yaxis.tick_left()
ax.yaxis.tick_right()
ax.set_xticklabels(["0 (No conflict)","1 (Conflict)"])
ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')
ax2.set_ylabel('Frequency [%]')
for p in ax.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        ax.annotate('{:.2f}%'.format(100.*y/total), (x.mean(), y), 
                ha='center', va='bottom') 
ax2.set_ylim(0,100)
ax.set_ylim(0,total)
ax2.grid(None)
plt.show()

In [ ]:
#ISIKLIKKU LÄHENEMIST VAJAVAD TULBAD

#muudame nende tulpade sisu
data1[["Amino_acids_initial", "Amino_acids_replacement"]] = data1.Amino_acids.str.split("/",expand=True)
data1[["Codons_initial", "Codons_replacement"]] = data1.Codons.str.split("/",expand=True)

data1["EXON"]=data1["EXON"].astype(str)
data1["INTRON"]=data1["INTRON"].astype(str)

#data1["EXON"].isnull().sum()
data1.loc[data1["EXON"] == data1["INTRON"], ["INTRON", "EXON"]] = np.nan
data1.loc[(data1["EXON"]!='nan') & (data1["INTRON"]!='nan') & (data1["REF"].str.len() == 1), ["INTRON", "EXON"]] = np.nan

#viskame välja kõik nan valued
data1 = data1[data1['EXON'].notna()]
data1 = data1[data1['INTRON'].notna()]

data1[["EXON_start", "EXON_end"]] = data1.EXON.str.split("/",expand=True)
data1[["INTRON_start", "INTRON_end"]] = data1.INTRON.str.split("/",expand=True)

##CHROM
data1["CHROM"]=data1["CHROM"].astype(str)
data1.loc[data1["CHROM"] == "X", "CHROM"] = "23"
data1.loc[data1["CHROM"] == "MT", "CHROM"] = "24"

In [ ]:
CLNVCdata = deepcopy(data1)
CLNVCdata = CLNVCdata[~CLNVCdata['CLNVC'].isin(['single_nucleotide_variant'])]

In [ ]:
fig = plt.figure(figsize = (20, 10))
ax = sns.countplot(x= 'CLASS', data = CLNVCdata, hue = 'CLNVC',palette="rocket")
total=len(data1)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.3f}%'.format(100*height/total),
            ha="center",weight="bold",fontsize=15) 


ax.set_xticklabels(['0 (No conflict)','1 (Conflict)'],fontsize=15, weight="bold")
ax.legend(fontsize=20, title="Variant type", title_fontsize=20)
ax.set_xlabel("Classification",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.show()

In [ ]:
#muudame nende tulpade sisu
data1[["Amino_acids_initial", "Amino_acids_replacement"]] = data1.Amino_acids.str.split("/",expand=True)
data1[["Codons_initial", "Codons_replacement"]] = data1.Codons.str.split("/",expand=True)

data1["EXON"]=data1["EXON"].astype(str)
data1["INTRON"]=data1["INTRON"].astype(str)

#data1["EXON"].isnull().sum()
data1.loc[data1["EXON"] == data1["INTRON"], ["INTRON", "EXON"]] = np.nan
data1.loc[(data1["EXON"]!='nan') & (data1["INTRON"]!='nan') & (data1["REF"].str.len() == 1), ["INTRON", "EXON"]] = np.nan

#viskame välja kõik nan valued
data1 = data1[data1['EXON'].notna()]
data1 = data1[data1['INTRON'].notna()]

data1[["EXON_start", "EXON_end"]] = data1.EXON.str.split("/",expand=True)
data1[["INTRON_start", "INTRON_end"]] = data1.INTRON.str.split("/",expand=True)

data1["EXON_start"]=data1["EXON_start"].astype(float)
data1["EXON_end"]=data1["EXON_end"].astype(float)
data1["INTRON_start"]=data1["INTRON_start"].astype(float)
data1["INTRON_end"]=data1["INTRON_end"].astype(float)

data1["CHROM"]=data1["CHROM"].astype(str)
data1.loc[data1["CHROM"] == "X", "CHROM"] = "23"
data1.loc[data1["CHROM"] == "MT", "CHROM"] = "24"

#LIHTSASTI DEFINEERITAVAD TULBAD
inttüüpi = ["POS", "ORIGIN", "CLASS", "CHROM"]
floattüüpi = ["AF_ESP", "AF_EXAC", "AF_TGP", "LoFtool", "CADD_PHRED", "CADD_RAW" ,  "STRAND", "BLOSUM62","EXON_start", "EXON_end","INTRON_start", "INTRON_end"]
strtüüpi = ["REF", "ALT", "Consequence", "IMPACT", "SYMBOL", "BIOTYPE" , "CLNVC", "IMPACT", "SIFT", "PolyPhen", "Amino_acids_initial", "Amino_acids_replacement", "Codons_initial", "Codons_replacement", "cDNA_position" , "CDS_position", "Protein_position"]


def datatypes(inttüüpi, strtüüpi, floattüüpi):
  for i in range(len(inttüüpi)):
      data1[inttüüpi[i]]=data1[inttüüpi[i]].astype(int)

  for i in range(len(strtüüpi)):
      data1[strtüüpi[i]]=data1[strtüüpi[i]].astype(str)
  
  for i in range(len(floattüüpi)):
      data1[floattüüpi[i]]=data1[floattüüpi[i]].astype(float).round(5)
      
datatypes(inttüüpi, strtüüpi, floattüüpi)

data1.head()


In [ ]:
#CDNA, CDS ja protein tulpade töötlus
data1[["cDNA_pos_start" , "cDNA_pos_end"]] = data1.cDNA_position.str.split("-",expand=True)
data1[["CDS_pos_start" , "CDS_pos_end"]] = data1.CDS_position.str.split("-",expand=True)
data1[["Protein_pos_start" , "Protein_pos_end"]] = data1.Protein_position.str.split("-",expand=True)

floatiks_ja_nan_nimetada = ["cDNA_pos_start" , "cDNA_pos_end", "CDS_pos_start" , "CDS_pos_end", "Protein_pos_start" , "Protein_pos_end"]
for i in range(len(floatiks_ja_nan_nimetada)):
  data1.loc[data1[floatiks_ja_nan_nimetada[i]] == "?", floatiks_ja_nan_nimetada[i]] = "999999999"
    
for i in range(len(floatiks_ja_nan_nimetada)):
  data1[floatiks_ja_nan_nimetada[i]]=data1[floatiks_ja_nan_nimetada[i]].astype(float)
  data1.loc[data1[floatiks_ja_nan_nimetada[i]].isna(), floatiks_ja_nan_nimetada[i]] = 999999999.0

In [ ]:
välja = ["cDNA_position","CDS_position","Protein_position","Allele", "Amino_acids", "Codons", "EXON","INTRON", "CLNDN", "CLNDISDB","CLNDNINCL", "CLNSIGINCL", "CLNDISDBINCL", "CLNHGVS", "CLNVI", "MC", "SSR", "Feature_type", "Feature", "DISTANCE", "BAM_EDIT", "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE"]
for i in välja:
    data1 = data1.drop(labels=[i], axis=1)

data1.describe()

In [ ]:
#asendame nendes tulpades nan väärtused tulba keskmisega
keskmised=["LoFtool","CADD_PHRED","CADD_RAW","BLOSUM62"]
for i in range(len(keskmised)):
    data1.loc[data1[keskmised[i]].notna()==False, keskmised[i]] = data1[keskmised[i]].mean()

#asendame nendes tulpades nan väärtused 0-ga
for i in range(len(floattüüpi)):
    data1.loc[data1[floattüüpi[i]].notna()==False, floattüüpi[i]] = 0
        
data1.AF_ESP[data1.AF_ESP == 0].count()

In [ ]:
mitteInt = floattüüpi + strtüüpi

valja = ["cDNA_position" , "CDS_position", "Protein_position"]
for i in valja:
    mitteInt.remove(i)

for i in range(len(mitteInt)):
    if data1[mitteInt[i]].isnull().values.any():
        print(mitteInt[i])

In [ ]:
data1.shape

In [ ]:
#onehotencoding
data1 = pd.get_dummies(data1,columns=["REF", "ALT", "CLNVC", "Consequence", "IMPACT", "SYMBOL", "BIOTYPE","SIFT",
                                      "PolyPhen","Amino_acids_initial", 
                                      "Amino_acids_replacement","Codons_initial","Codons_replacement"], dummy_na=False)

In [ ]:
print(len(data1.columns))
tulbad = list(data1.columns)
for i in range(len(tulbad)):
    if "_nan" in tulbad[i]:
        data1 = data1.drop(tulbad[i], axis=1)
        print(tulbad[i])

print(len(data1.columns))

In [ ]:
tulbad = list(data1.columns)
for i in range(len(tulbad)):
    if data1[tulbad[i]].isnull().values.any():
        print(tulbad[i])
        
data1.isnull().sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data1.drop(labels='CLASS',axis=1),data1.CLASS,train_size=0.7,random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=3).fit(X_train, y_train)

y_pred = dt.predict(X_test)
acc_score=accuracy_score(y_pred, y_test)
print("Accuracy score:\n%s " %round(acc_score*100,2))

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = dt.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)


plt.title('DecisionTreeClassifier ROC')
plt.plot(fpr, tpr, 'r', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'b--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#mudeli fittimine

rf = RandomForestClassifier(n_estimators=100, random_state=69,
                            oob_score=True).fit(X_train, y_train)
y_pred = rf.predict(X_test)
acc_score=accuracy_score(y_pred, y_test)
print("Accuracy score:\n%s " %round(acc_score*100,2))

In [ ]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(
        max_depth=9, n_estimators=100,random_state=0,num_leaves=70,min_data_in_leaf=50)

lgbm.fit(X_train, y_train)

y_pred = lgbm.predict(X_test)
acc_score=accuracy_score(y_pred, y_test)
print("Accuracy score:\n%s " %round(acc_score*100,2))

In [ ]:
probs = lgbm.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)


plt.title('LGBMClassifier ROC')
plt.plot(fpr, tpr, 'r', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'b--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#aitab välja praakida ebavajalikud featurid

feat_importances = pd.Series(rf.feature_importances_, index=data1.columns[:-1])
feat_importances.nlargest(15).plot(kind='barh')
plt.title("Top 15 important features")
plt.show()

In [ ]:
#checking for overfiting
print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(X_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(X_test, y_test)))

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
data1.shape

In [ ]:
#pairwise correlation graph

top15 = data1[["AF_EXAC", "CADD_PHRED", "POS", "AF_TGP", "LoFtool", "AF_ESP", "INTRON_end", "cDNA_pos_end", "CDS_pos_end",
              "STRAND", "BLOSUM62", "CHROM", "EXON_start", "EXON_end", "CADD_RAW"]]

fig, ax = plt.subplots(figsize=(10,7))
corr = top15.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

In [ ]:
corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot

# define the model
model = LinearRegression()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.coef_

In [ ]:
data2=pd.DataFrame(columns=['Feature_name', 'Score'])

score = []
data1Tulbad = []

for i,v in enumerate(importance):
    data1Tulbad.append(list(data1.columns)[i])
    score.append(v)

data2['Feature_name'] = data1Tulbad
data2["Score"] = score

data2.head()

In [ ]:
data2.Feature_name[data2.Score.idxmax()]

In [ ]:
data2["Score"]=data2["Score"].astype(float)
data2=data2.sort_values(by=["Score"],ascending=False)

print(data2.head(15))
print()
print(data2.tail(15))

In [ ]:
#pairwise correlation graph

absValues = deepcopy(data2)
absValues.Score = absValues.Score.abs()
absValues=absValues.sort_values(by=["Score"],ascending=False)


absValues = absValues[:10]


In [ ]:
fig = plt.figure(figsize = (10, 5))
bar = sns.barplot(x= absValues["Score"], y=absValues['Feature_name'],palette="hls")
bar.set_yticklabels(absValues['Feature_name'], fontsize=12, weight="bold")
bar.set_xlabel("Score",fontsize=20)
plt.title("Features\n", fontsize=25)